In [1]:
import pandas as pd
import numpy
from __future__ import division

In [2]:
df3_WS3T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/pythonProject/segment3_WS3_has_duration.csv")

In [3]:
df3_WS3T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,...,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS,time_used_str,time_used_second
0,1161800|550469|842233,Assessment Outcome - Not Successful,Assessment Outcome - Not Successful,14/12/2019 14:24:13,KRIT,Faculty,1,Y,2. Perform Assessment,Part 3 - Assessment Outcome,...,MCW,HDCW,Full Degree,AF,AF,NaN,NaN,N,NaN,0.0


In [4]:
numOfWorkflowProcesses = df3_WS3T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic


In [5]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [6]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [7]:
list_avgtime_per_key =[]
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df3_WS3T[df3_WS3T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))

list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [8]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df3_WS3T[df3_WS3T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [12]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df3_WS3T[df3_WS3T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED']=='Release Offer']
        count_df_id += df_id['CASE_ID'].count()
        count_df_end_step += df_end_step['CASE_ID'].count()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")

In [13]:
s3_t3=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end})
s3_t3.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment
0,1,100,0 days 0 hours 0 minutes 0 seconds,0.0 days 0.0 hours 0.0 minutes 0.0 seconds,0.0%
1,2,18,5 days 20 hours 7 minutes 13 seconds,0.0 days 0.0 hours 3.0 minutes 57.0 seconds,0.0%
2,3,215,1 days 6 hours 54 minutes 45 seconds,0.0 days 0.0 hours 2.0 minutes 30.0 seconds,31.63%
3,4,39,7 days 22 hours 19 minutes 58 seconds,0.0 days 0.0 hours 2.0 minutes 55.0 seconds,20.51%
4,5,5,4 days 15 hours 55 minutes 6 seconds,4.0 days 23.0 hours 37.0 minutes 6.0 seconds,20.0%
5,6,4,7 days 1 hours 40 minutes 40 seconds,6.0 days 14.0 hours 52.0 minutes 40.0 seconds,8.33%
6,7,2,8 days 22 hours 22 minutes 34 seconds,8.0 days 22.0 hours 22.0 minutes 34.5 seconds,14.29%
7,8,2,7 days 21 hours 14 minutes 42 seconds,7.0 days 21.0 hours 14.0 minutes 42.0 seconds,12.5%
8,9,2,7 days 0 hours 29 minutes 35 seconds,7.0 days 0.0 hours 29.0 minutes 35.0 seconds,5.56%
9,10,1,21 days 22 hours 36 minutes 50 seconds,21.0 days 22.0 hours 36.0 minutes 50.0 seconds,10.0%


In [11]:
s3_t3.to_csv('segment3_WS3_table.csv', index=False)



